# Đánh giá mô hình

### 1. Import thư viện và đường đẫn

In [ ]:
import joblib
import os
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

# Đường dẫn đến các mô hình đã lưu
models_dir = "../models"
model_files = ["logistic_imbalance.pkl", 
               "decision_tree_imbalance.pkl", 
               "random_forest_imbalance.pkl", 
               "xgboost_imbalance.pkl",
               "logistic_smote.pkl", 
               "decision_tree_smote.pkl", 
               "random_forest_smote.pkl", 
               "xgboost_smote.pkl",
               "logistic_adasyn.pkl", 
               "decision_tree_adasyn.pkl", 
               "random_forest_adasyn.pkl", 
               "xgboost_adasyn.pkl"]

# Đường dẫn dữ liệu kiểm tra
test_data_path = "../data/splits/X_test.csv"
test_labels_path = "../data/splits/y_test.csv"

figure_dir = "../reports/figures"



### 2. Đọc tập dữ liệu và mô hình

In [ ]:
# Tạo dictionary để lưu các mô hình đã đọc
models = {}

# Đọc các mô hình
for model_file in model_files:
    model_path = os.path.join(models_dir, model_file)
    if os.path.exists(model_path):
        model_name = model_file.split(".pkl")[0]
        models[model_name] = joblib.load(model_path)
        print(f"Đã tải mô hình: {model_name}")
    else:
        print(f"Mô hình không tồn tại: {model_path}")

# Đọc dữ liệu kiểm tra
X_test = pd.read_csv(test_data_path)
y_test = pd.read_csv(test_labels_path).values.ravel()


### 3. Tính toán các chỉ số đánh giá

In [ ]:
# DataFrame để lưu kết quả đánh giá
evaluation_results = pd.DataFrame()

# Đánh giá từng mô hình
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Tính các chỉ số
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
    
    # Lưu kết quả vào DataFrame
    data_type = model_name.split("_")[-1]  # imbalance, smote, adasyn
    model_type = "_".join(model_name.split("_")[:-1])  # logistic_regression, decision_tree, etc.
    new_result = [model_type, data_type, accuracy, precision, recall, f1, roc_auc]
    evaluation_results = pd.concat([evaluation_results, pd.DataFrame([new_result])], ignore_index=True)
    
evaluation_results.columns = ["Model", "Data Type", "Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"]
# Hiển thị kết quả
print(evaluation_results)
 

### 4. Lưu các chỉ số đánh giá

In [ ]:
# Lưu bảng kết quả đánh giá ra tệp CSV
evaluation_results.to_csv("../reports/tables/evaluation_results.csv", index=False)


### 5. Vẽ các biểu đồ

#### Chuyển dữ liệu thành định dạng dài (long format)

In [ ]:
# Chuyển đổi dataframe sang định dạng dài
evaluation_result_long = evaluation_results.melt(id_vars=['Model', 'Data Type'],
                                                value_vars=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC'],
                                                var_name='Metric',
                                                value_name='Score')


#### Vẽ biểu đồ cột so sánh các mô hình và các chỉ số

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='Model', y='Score', hue='Metric', data=evaluation_result_long[evaluation_result_long['Data Type'] == 'imbalance'], ci=None)

# Cài đặt tiêu đề và nhãn
plt.title('So sánh hiệu suất mô hình trên các bộ dữ liệu imbalance')
plt.ylabel('Score')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.legend(title="Metrics", loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

plt.savefig(os.path.join(figure_dir, "15_comparison_imbalance.png"))

# Hiển thị biểu đồ
plt.show()


In [ ]:
# Vẽ biểu đồ cột so sánh các mô hình và các chỉ số
plt.figure(figsize=(12, 8))
sns.barplot(x='Model', y='Score', hue='Metric', data=evaluation_result_long[evaluation_result_long['Data Type'] == 'smote'], ci=None)

# Cài đặt tiêu đề và nhãn
plt.title('So sánh hiệu suất mô hình trên các bộ dữ liệu smote')
plt.ylabel('Score')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.legend(title="Metrics", loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

plt.savefig(os.path.join(figure_dir, "16_comparison_smote.png"))

# Hiển thị biểu đồ
plt.show()


In [ ]:
# Vẽ biểu đồ cột so sánh các mô hình và các chỉ số
plt.figure(figsize=(12, 8))
sns.barplot(x='Model', y='Score', hue='Metric', data=evaluation_result_long[evaluation_result_long['Data Type'] == 'adasyn'], ci=None)

# Cài đặt tiêu đề và nhãn
plt.title('So sánh hiệu suất mô hình trên các bộ dữ liệu adasyn')
plt.ylabel('Score')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.legend(title="Metrics", loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

plt.savefig(os.path.join(figure_dir, "17_comparison_adasyn.png"))

# Hiển thị biểu đồ
plt.show()


#### Vẽ biểu đồ ROC AUC

In [ ]:
def build_measure_model(models):
    plt.figure(figsize=(12, 6))

    for name, model, X_test, y_test in models:
        
        # Predict
        y_test_pred = model.predict(X_test)
        
        # plot ROC Curve
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_pred)
        auc = metrics.roc_auc_score(y_test, y_test_pred)
        plt.plot(fpr, tpr, linewidth=2, label=name + ", auc=" + str(auc))

    plt.legend(loc=4)
    plt.plot([0, 1], [0, 1], 'k--' )
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for Predicting a credit card fraud detection')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    
    # Lấy tên từ model đầu tiên
    model_names = "_".join([name.split(" ")[0] for name, _, _, _ in models])
    save_path = os.path.join(figure_dir, f"roc_curve_{model_names}.png")
    print(save_path)
    plt.savefig(save_path)
    plt.show()


#### Vẽ cho Logistic regression

In [ ]:
LRmodels = [] #LogisticRegression model

LRmodels.append(('LR imbalance', models["logistic_imbalance"], X_test, y_test))
LRmodels.append(('LR SMOTE', models["logistic_smote"], X_test, y_test))
LRmodels.append(('LR ADASYN', models["logistic_adasyn"], X_test, y_test))

build_measure_model(LRmodels)

#### Vẽ cho Decision tree

In [ ]:
DTmodels = [] #Decision Tree model

DTmodels.append(('DT imbalance', models["decision_tree_imbalance"], X_test, y_test))
DTmodels.append(('DT SMOTE', models["decision_tree_smote"], X_test, y_test))
DTmodels.append(('DT ADASYN', models["decision_tree_adasyn"], X_test, y_test))

build_measure_model(DTmodels)

#### Vẽ cho Random forest

In [ ]:
RFmodels = [] #Random Forest model

RFmodels.append(('RF imbalance', models["random_forest_imbalance"],X_test,y_test))
RFmodels.append(('RF SMOTE', models["random_forest_smote"], X_test, y_test))
RFmodels.append(('RF ADASYN', models["random_forest_adasyn"], X_test, y_test))

build_measure_model(RFmodels)

#### Vẽ cho XGBoost 

In [ ]:
XGBmodels = [] #XGBoost model

XGBmodels.append(('XGBoost imbalance', models["xgboost_imbalance"], X_test, y_test))
XGBmodels.append(('XGBoost SMOTE', models["xgboost_smote"], X_test, y_test))
XGBmodels.append(('XGBoost ADASYN', models["xgboost_adasyn"], X_test, y_test))

build_measure_model(XGBmodels)